In [285]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
import pymongo
from sqlalchemy import create_engine

In [261]:
pd.set_option('display.max_columns', None)

# Reading the data and performing basic operations to get an overview about the data set

In [262]:
df_orders = pd.read_csv('./Data-Superstore/Orders.csv')
df_returns = pd.read_csv('./Data-Superstore/Returns.csv')
df_users = pd.read_csv('./Data-Superstore/Users.csv')

In [263]:
df_orders.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,Small Box,Avery 49,0.36,Central,Illinois,Addison,60101,5/28/2012,5/30/2012,1.3200,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,0.54,West,Washington,Anacortes,98221,7/7/2010,7/8/2010,4.5600,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Paper,Small Box,Xerox 1968,0.37,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-47.6400,7,49.92,88523
3,23087,Not Specified,0.01,5.68,3.60,3,Bonnie Potter,Regular Air,Corporate,Office Supplies,"Scissors, Rulers and Trimmers",Small Pack,Acme® Preferred Stainless Steel Scissors,0.56,West,Washington,Anacortes,98221,7/27/2011,7/28/2011,-30.5100,7,41.64,88523
4,23088,Not Specified,0.00,205.99,2.50,3,Bonnie Potter,Express Air,Corporate,Technology,Telephones and Communication,Small Box,V70,0.59,West,Washington,Anacortes,98221,7/27/2011,7/27/2011,998.2023,8,1446.67,88523


In [207]:
df_returns.head()

,Order ID,Status
0,65,Returned
1,612,Returned
2,614,Returned
3,678,Returned
4,710,Returned


In [208]:
df_users.head()

,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [209]:
df_orders.shape

(9426, 24)

In [210]:
df_returns.shape

(1634, 2)

In [211]:
df_users.shape

(4, 2)

In [212]:
df_orders.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

In [213]:
df_returns.columns

Index(['Order ID', 'Status'], dtype='object')

In [214]:
df_orders.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Row ID,9426.0,20241.015277,6101.890965,2.000,19330.250000,21686.5000,24042.75000,26399.000
Discount,9426.0,0.049628,0.031798,0.000,0.020000,0.0500,0.08000,0.250
Unit Price,9426.0,88.303686,281.540982,0.990,6.480000,20.9900,85.99000,6783.020
Shipping Cost,9426.0,12.795142,17.181203,0.490,3.192500,6.0500,13.99000,164.730
Customer ID,9426.0,1738.422236,979.167197,2.000,898.000000,1750.0000,2578.75000,3403.000
Product Base Margin,9354.0,0.512189,0.135229,0.350,0.380000,0.5200,0.59000,0.850
Postal Code,9426.0,52446.327286,29374.597802,1001.000,29406.000000,52302.0000,78516.00000,99362.000
Profit,9426.0,139.236410,998.486483,-16476.838,-74.017375,2.5676,140.24385,16332.414
Quantity ordered new,9426.0,13.798430,15.107688,1.000,5.000000,10.0000,17.00000,170.000
Sales,9426.0,949.706272,2598.019818,1.320,61.282500,203.4550,776.40250,100119.160


In [215]:
df_orders.columns

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')

## Working with mysql

In [216]:
def create_connection(host_name, user_name, password, database):
    
    try:
        connection = mysql.connector.connect(host=host_name, user = user_name, password=password, use_pure=True, auth_plugin='mysql_native_password')
    except Error as err:
        print('Error : ',err)
    else:
        return connection
        
def execute_query(connection, query:str,commit:bool=False):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        if commit:
            connection.commit()
        print("Query was successfully executed")
    except Error as err:
        print('Error : ',err)
    else:
        return result
    

def close_connection(connection):
    # cursor.close()
    connection.close()

In [217]:
host_name = 'localhost'
user_name = 'root'
password = '<password>'

conn = create_connection(host_name, user_name, password, 'ineuron_data')

print(conn)

In [260]:
close_connection(conn)

In [259]:
engine = create_engine("mysql+pymysql://root:<password>@localhost:3306/ineuron_data")

all_tables = [df_returns,df_users,df_orders]
table_names = ['returns','users','orders']

for index in range(len(all_tables)):
    try:
        all_tables[index].to_sql(name=table_names[index], con=engine, index=False, if_exists='replace')
    except Exception as e:
        print("Error is: ", e)
    else:
        print("Query is Successful")

Query is Successful
Query is Successful
Query is Successful


In [264]:
df_orders.head(2)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,Small Box,Avery 49,0.36,Central,Illinois,Addison,60101,5/28/2012,5/30/2012,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,0.54,West,Washington,Anacortes,98221,7/7/2010,7/8/2010,4.56,4,13.01,88522


## Upload data into MongoDb

> Convert files into json format

In [268]:

# df_orders['Ship Date'] = pd.to_datetime(df_orders['Ship Date'])
# df_orders['Order Date'] = pd.to_datetime(df_orders['Order Date'])

In [267]:
df_orders['Order Date'] = df_orders['Order Date'].dt.strftime('%Y-%m-%d')
df_orders['Ship Date'] = df_orders['Ship Date'].dt.strftime('%Y-%m-%d')

df_orders.to_json('./orders.json',date_format='iso')
df_returns.to_json('./returns.json',date_format='iso')
df_users.to_json('./users.json',date_format='iso')

df_orders['Ship Date'] = pd.to_datetime(df_orders['Ship Date'])
df_orders['Order Date'] = pd.to_datetime(df_orders['Order Date'])

In [273]:
orders_json = pd.read_json('./orders.json')
users_json = pd.read_json('./users.json')
returns_json = pd.read_json('./returns.json')

orders_json.head(3)

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Name,Product Base Margin,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,18606,Not Specified,0.01,2.88,0.50,2,Janice Fletcher,Regular Air,Corporate,Office Supplies,Labels,Small Box,Avery 49,0.36,Central,Illinois,Addison,60101,2012-05-28,2012-05-30,1.32,2,5.90,88525
1,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Pens & Art Supplies,Wrap Bag,SANFORD Liquid Accent™ Tank-Style Highlighters,0.54,West,Washington,Anacortes,98221,2010-07-07,2010-07-08,4.56,4,13.01,88522
2,23086,Not Specified,0.03,6.68,6.15,3,Bonnie Potter,Express Air,Corporate,Office Supplies,Paper,Small Box,Xerox 1968,0.37,West,Washington,Anacortes,98221,2011-07-27,2011-07-28,-47.64,7,49.92,88523


In [274]:
users_json.head()

,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


In [275]:
returns_json.head()

,Order ID,Status
0,65,Returned
1,612,Returned
2,614,Returned
3,678,Returned
4,710,Returned


> Changing Index into String (from int)

In [283]:
orders_json.index = orders_json.index.map(str)
users_json.index = users_json.index.map(str)
returns_json.index = returns_json.index.map(str)

> Changing JSOn file into a Dictionary

In [284]:
orders_dict = orders_json.to_dict('index')
users_dict = users_json.to_dict('index')
returns_dict = returns_json.to_dict('index')

> Connecting to MongoDb

In [286]:
# CONNECTING TO MONGODB ATLAS
try:
    client = pymongo.MongoClient("mongodb+srv://<password>:ineuron@cluster0.ex1ru.mongodb.net/?retryWrites=true&w=majority")
except Exception as e:
    print("Error => {e}")
db = client.test

> Creating Database(Superstore) and collections (users, orders and returns)

In [287]:
try:
    database = client['superstore']
    collection_orders = database['orders']
    collection_users = database['users']
    collection_returns = database['returns']
except Exception as e:
    print("Error in creating database and collection [dress,sales]")
    print(f"Error => {e}")

> Inserting data into the ccllections

In [288]:
collection_orders.insert_one(orders_dict)

In [289]:
collection_returns.insert_one(returns_dict)

In [290]:
collection_users.insert_one(users_dict)

# Find out how much return we have recived and with a product id 

In [219]:
df_returns['Status'].value_counts()

Returned    1634
Name: Status, dtype: int64

In [220]:
df_returns.shape

(1634, 2)

In [221]:
returned_products = pd.merge(right=df_returns, left=df_orders, how='inner', left_on='Order ID', right_on='Order ID')['Row ID']

In [222]:
returned_products

0     1359
1     1950
2     1951
3     5055
4     5573
      ... 
93     986
94     987
95    6041
96    1910
97    6625
Name: Row ID, Length: 98, dtype: int64

### Total products returned are 98

# Try to join order and ship date both in sql and pandas 

# Try to find out how many unique customer that we have 

In [223]:
unique_customers = df_orders.groupby('Customer ID')['Customer Name'].value_counts()

In [224]:
np.size(unique_customers)

2703

> We have a total of 2703 Unique Customers

# Try to find out in how many regions we are selling a product and who is a manager for a respective region 

In [225]:
df_users

,Region,Manager
0,Central,Chris
1,East,Erin
2,South,Sam
3,West,William


# find out how many different differnet shipement mode that we have and what is a percentage usablity of all the shipment mode with respect to dataset 

In [226]:
shipmnet_modes = df_orders['Ship Mode'].value_counts().reset_index()
shipmnet_modes.columns = ['Shipment Mode','No of Shipments']
shipmnet_modes

,Shipment Mode,No of Shipments
0,Regular Air,7036
1,Delivery Truck,1283
2,Express Air,1107


In [227]:
total_shipments = df_orders['Ship Mode'].size
total_shipments

9426

In [228]:
shipmnet_modes['usage_percentage'] = ((shipmnet_modes['No of Shipments'])/total_shipments)*100

In [229]:
shipmnet_modes

,Shipment Mode,No of Shipments,usage_percentage
0,Regular Air,7036,74.644600
1,Delivery Truck,1283,13.611288
2,Express Air,1107,11.744112


# Create a new coulmn and try to find our a diffrence between order date and shipment date

In [230]:
df_orders['Ship Date'] = pd.to_datetime(df_orders['Ship Date'])
df_orders['Order Date'] = pd.to_datetime(df_orders['Order Date'])

In [231]:
df_orders['Ship Date'] - df_orders['Order Date']

0      2 days
1      1 days
2      1 days
3      1 days
4      0 days
        ...  
9421   1 days
9422   0 days
9423   2 days
9424   3 days
9425   2 days
Length: 9426, dtype: timedelta64[ns]

In [232]:
df_orders['diff_dates'] = (df_orders['Ship Date'] - df_orders['Order Date']).dt.days

In [233]:
df_orders.tail()['diff_dates']

9421    1
9422    0
9423    2
9424    3
9425    2
Name: diff_dates, dtype: int64

# Based on above question, find out for which order id had shipment duration more than 10 days 

In [234]:
shipmnet_more_than_10_days = df_orders.query('diff_dates >= 10')[['Order ID','diff_dates']].reset_index()

In [235]:
shipmnet_more_than_10_days

,index,Order ID,diff_dates
0,643,87215,84
1,1548,86318,11
2,1549,86318,15
3,1678,87957,17
4,1679,87957,11
5,1680,87957,28
6,1697,19556,17
7,1698,19556,11
8,1699,19556,28
9,2515,86177,92


In [236]:
# df_orders[df_orders['Order ID']==87215]
# df_returns[df_returns['Order ID']==87215]

# Try to find out a list of a returned order which sihpment duration was more then 15 days and find out that region manager as well 

In [237]:
returned_products

0     1359
1     1950
2     1951
3     5055
4     5573
      ... 
93     986
94     987
95    6041
96    1910
97    6625
Name: Row ID, Length: 98, dtype: int64

In [238]:
order_ids_with_15_days_diff = pd.merge(right=df_users, left=df_orders, how='inner', left_on='Region', right_on='Region')[['Row ID','Manager', 'diff_dates']]

In [239]:
order_ids_with_15_days_diff[order_ids_with_15_days_diff['Row ID']==1910]

,Row ID,Manager,diff_dates
7236,1910,Erin,2


In [240]:
pd.merge(left=returned_products, right=order_ids_with_15_days_diff, how='inner', left_on='Row ID', right_on='Row ID').query('diff_dates > 15')

,Row ID,Manager,diff_dates


    - From above query, we can say that No order was returned having Difference in days more that 15 between Order Date and Shipment Date

# Gorup by region and find out which region is more profitable 

In [241]:
df_orders.groupby("Region").aggregate({'Profit':'sum'}).sort_values(by='Profit', ascending=False)

,Profit
Region,
Central,519825.567066
East,377566.186045
West,310849.453891
South,104201.192421


> Central Region is most profitable

# Try to find out overall in which country we are giving more discount 

In [242]:
df_orders.groupby('City').aggregate({'Discount':'sum'}).sort_values(by=['Discount'], ascending=False)

,Discount
City,
Los Angeles,10.37
New York City,10.16
Chicago,4.83
Seattle,4.47
Boston,3.79
...,...
Billerica,0.00
North Kingstown,0.00
Toms River,0.00


In [243]:
df_orders.groupby('State or Province').aggregate({'Discount':'sum'}).sort_values(by=['Discount'], ascending=False)

,Discount
State or Province,
California,52.28
Texas,31.36
Illinois,29.71
New York,28.86
Florida,26.38
Ohio,19.71
Washington,15.48
Michigan,15.29
Pennsylvania,13.32


# Give me a list of unique postal code

In [244]:
postal_codes = df_orders['Postal Code'].unique()

In [245]:
postal_codes.size

1697

In [246]:
postal_codes_list = list(postal_codes)
# postal_codes_list

# which customer segement is more profitalble find it out .

In [247]:
df_orders.groupby(['Customer Segment']).aggregate({'Profit':'sum'}).sort_values(by=['Profit'], ascending=False)

,Profit
Customer Segment,
Corporate,505538.627783
Small Business,316474.592482
Home Office,283869.553809
Consumer,206559.625349


# Try to find out the 10th most loss making product category

In [248]:
df_orders.groupby(['Product Category']).aggregate({'Profit':'sum'}).sort_values('Profit')

,Profit
Product Category,
Furniture,177354.298188
Office Supplies,451990.216493
Technology,683097.884742


In [249]:
df_orders['Product Category'].value_counts()

Office Supplies    5181
Technology         2312
Furniture          1933
Name: Product Category, dtype: int64

In [250]:
top_10_loss_making_products = df_orders.groupby('Product Sub-Category').aggregate({'Profit':'sum'}).sort_values(by='Profit').reset_index()[:10]

In [251]:
top_10_loss_making_products

,Product Sub-Category,Profit
0,Tables,-72495.061875
1,Bookcases,-7708.748564
2,Rubber Bands,-2841.722459
3,"Scissors, Rulers and Trimmers",-1936.849120
4,Pens & Art Supplies,1195.902840
5,Storage & Organization,8078.804728
6,Labels,17775.320505
7,Paper,35361.621738
8,Envelopes,46133.223840
9,Computer Peripherals,87917.842513


# Try to find out 10 top  product with highest margins

In [252]:
# product_margins = df.groupby(['Product Name']).aggregate({'Product Base Margin':'sum','Product Category':'count'})
product_margins = df_orders.groupby(['Product Sub-Category']).aggregate({'Product Base Margin':'sum','Product Category':'count'})
product_margins.columns = ['Product Base Margin Total','Product Base Margin Count']
product_margins['Product Margin'] = product_margins['Product Base Margin Total']/product_margins['Product Base Margin Count']
product_margins

,Product Base Margin Total,Product Base Margin Count,Product Margin
Product Sub-Category,,,
Appliances,273.00,492,0.554878
Binders and Binder Accessories,384.57,1028,0.374095
Bookcases,132.67,206,0.644029
Chairs & Chairmats,260.59,440,0.592250
Computer Peripherals,500.91,846,0.592092
Copiers and Fax,41.26,98,0.421020
Envelopes,101.66,272,0.373750
Labels,123.97,329,0.376809
Office Furnishings,463.23,883,0.524609


In [253]:
product_margins.sort_values(by='Product Margin', ascending=False)[:10]

,Product Base Margin Total,Product Base Margin Count,Product Margin
Product Sub-Category,,,
Tables,271.19,404,0.671262
Bookcases,132.67,206,0.644029
"Scissors, Rulers and Trimmers",99.56,155,0.642323
Storage & Organization,390.26,610,0.639770
Chairs & Chairmats,260.59,440,0.592250
Computer Peripherals,500.91,846,0.592092
Telephones and Communication,576.76,992,0.581411
Appliances,273.00,492,0.554878
Pens & Art Supplies,384.57,721,0.533384


In [293]:
from IPython.display import Image
  
# get the image
Image(url="./orders_ss.png", width=1000, height=500)

In [294]:
Image(url="./returns_ss.png", width=1000, height=500)

## MYSQL Images

In [296]:
Image(url="./orders_sql_ss.png", width=1000, height=500)

In [297]:
Image(url="./users_sql_ss.png", width=1000, height=500)